# B25 - Training

In [1]:
from gensim.models import Word2Vec
import pickle

In [2]:
with open('../data/tokenized_data/playlist_names/dataset_train.pkl', 'rb') as f:
    train_dataset = pickle.load(f)

In [ ]:
word2vec_model = Word2Vec(sentences=train_dataset, workers=10, vector_size=50, window=5, min_count=1, sg=0)
word2vec_model.save("../models/b25-sn-v50/b25-sn-v50.model")

In [ ]:
word2vec_model = Word2Vec(sentences=train_dataset, workers=10, vector_size=256, window=5, min_count=1, sg=0)
word2vec_model.save("../models/b25-sn-v256/b25-sn-v256-a.model")

In [ ]:
word2vec_model = Word2Vec(sentences=train_dataset, workers=10, vector_size=256, window=10, min_count=1, sg=0)
word2vec_model.save("../models/b25-sn-v256/b25-sn-v256-b.model")

In [ ]:
word2vec_model = Word2Vec(sentences=train_dataset, workers=10, vector_size=256, window=20, min_count=1, sg=0)
word2vec_model.save("../models/b25-sn-v256/b25-sn-v256-c.model")

In [ ]:
word2vec_model = Word2Vec(sentences=train_dataset, workers=10, vector_size=256, window=20, min_count=1, sg=1, ns_exponent=0.0)
word2vec_model.save("../models/b25-sn-v256/b25-sn-v256-d.model")

In [ ]:
word2vec_model = Word2Vec(sentences=train_dataset, workers=10, vector_size=512, window=100, min_count=1, sg=0)
word2vec_model.save("../models/b25-sn-v512/b25-sn-v512-a.model")

In [ ]:
word2vec_model = Word2Vec(sentences=train_dataset, workers=10, vector_size=512, window=100, min_count=1, sg=1, ns_exponent=0.0)
word2vec_model.save("../models/b25-sn-v512/b25-sn-v512-b.model")

In [ ]:
from song2vec import Song2Vec

model = Song2Vec(train_dataset[0:2000], vector_size=512, epochs=5, learning_rate=0.015)


model.save("../models/b25-sn-v512/b25-sn-v512-g.model")
print("Model saved successfully.")

# need some working with nan values in the dataset... or with nan values in the keys...

# 7.803 for 100
# 5670.13 * 7.803 that would be 30,7 Days

# programming it in c took it down to
# 5670.13 * 1.46 that would be 8278,38 min = 5,74 Days 

# generally it's an issues with floats in there
# need to cast every element of the list to string and also somehow ignore the nan values
# some data cleaning is needed to be done

# so i tried to let it train for 1593min and it did not finish...
# i need to optimize the whole thing way more to get some real numbers



In [ ]:
from algo_testing import song2vec_a

model = song2vec_a.Song2VecA(train_dataset[0:100], vector_size=512, epochs=5, learning_rate=0.005)

In [ ]:
model.nearest(train_dataset[0][0], 4)

In [4]:
from algo_testing import song2vec_c

model = song2vec_c.Song2VecC(train_dataset[0:5000], vector_size=512, epochs=30, learning_rate=0.015)

with open("../models/b25-sn-v512/b25-sn-v512-n.pkl", 'wb') as f:
    pickle.dump(model, f)

## Param

#### b25-sn-v50
> vector_size=50, window=5, min_count=1, sg=0

#### b25-sn-v256-a
> vector_size=256, window=5, min_count=1, sg=0

#### b25-sn-v256-b
> vector_size=256, window=10, min_count=1, sg=0

#### b25-sn-v256-c
> vector_size=256, window=20, min_count=1, sg=0

#### b25-sn-v256-d
> vector_size=256, window=20, min_count=1, sg=1, ns_exponent=0.0

#### b25-sn-v512-a
> vector_size=512, window=100, min_count=1, sg=0

#### b25-sn-v512-b
> vector_size=512, window=100, min_count=1, sg=1, ns_exponent=0.0

#### b25-sn-v512-c - CBOS
> vector_size=512, epochs=5, learning_rate=0.01

#### b25-sn-v512-d - CBOS
> vector_size=512, epochs=5, learning_rate=0.015